In [5]:
#rifatto in laboratorio
# rifatto con sathya il 20 ottobre
import json
import numpy
from sklearn.feature_extraction.text import CountVectorizer

testDataset=json.load(open('test.json'))
validDataset=json.load(open('valid.json'))

#combined_dataset=[]
#for diz in testDataset:
#    combined_dataset.append(diz['original_situation'])
#for diz in validDataset:
#    combined_dataset.append(diz['original_situation'])
#dataset combinati, numero di frasi
#len(combined_dataset)
#combined_dataset
i=0
combined_dataset={}
for diz in testDataset:
    combined_dataset[i]=diz
   # print(i)
    i=i+1
for diz in validDataset:
    combined_dataset[i]=diz
    i=i+1
#print(i)   
len(combined_dataset)

4613

In [6]:
#creo il vocabolario e il bow per le frasi
listaFrasi=[]
for diz in combined_dataset.values():
    listaFrasi.append(diz['original_situation'])
   # print(diz)
len(listaFrasi)
cv = CountVectorizer()
count_matrix = cv.fit(listaFrasi) #potevo anche non assegnarlo alla variabile count matrix
vocabulary_words = cv.get_feature_names()
print('Totale parole trovate = {}\n'.format(len(vocabulary_words)))
#print('lista di parole del vocabolario:\n\t{}\n'.format(vocabulary_words))

#creo il bow del dataset
frasi_BoW = cv.transform(listaFrasi)
print(frasi_BoW.shape)

Totale parole trovate = 5813

(4613, 5813)


In [7]:
#creo il vocabolario e il bow per le parole emotive
listaParoleEmotive=[]
for diz in combined_dataset.values():
    x=" ".join(diz['labels'])
    #print(x)
    listaParoleEmotive.append(x)
len(listaParoleEmotive)
pe_BoW=cv.transform(listaParoleEmotive)

print(pe_BoW.shape)

(4613, 5813)


In [8]:
newArray=[]
pe_BoW_array = pe_BoW.toarray()
for row in pe_BoW_array:
    riga=[]
    for i in row:
        if i>1:
            riga.append(1)
        else:
            riga.append(i)
    newArray.append(row)
    #print(row)
#newArray



In [9]:
#print('pe matrix:\n{}\n'.format(type(pe_BoW))
#print('frasi matrix :\n{}\n'.format(frasi_BoW))
print(pe_BoW[0])
print(vocabulary_words[4220])

  (0, 2166)	1
  (0, 2290)	1
  (0, 4220)	1
rendition


In [10]:
#importo lessico del modello
import pandas as pd
df=pd.read_csv(r'Ratings_Warriner_et_al.csv',usecols = ['Word','V.Mean.Sum','A.Mean.Sum'])
df.head()

,Word,V.Mean.Sum,A.Mean.Sum
0,aardvark,6.26,2.41
1,abalone,5.30,2.65
2,abandon,2.84,3.73
3,abandonment,2.63,4.95
4,abbey,5.85,2.20


In [11]:
df.iloc[[0]]

,Word,V.Mean.Sum,A.Mean.Sum
0,aardvark,6.26,2.41


In [12]:
valence=df['V.Mean.Sum']
arousal=df['A.Mean.Sum']
paroleVocabolarioInglese=df['Word']
type(valence)

pandas.core.series.Series

In [13]:
num_elemVal = valence.size
max_elemVal = valence.max()
min_elemVal = valence.min()
range_elemVal = max_elemVal - min_elemVal
mediaVal=valence.mean()

num_elemA = arousal.size
max_elemA = arousal.max()
min_elemA = arousal.min()
range_elemA = max_elemA - min_elemA
mediaA=arousal.mean()

In [14]:
print ('The number of valence samples is', num_elemVal,  num_elemA)
print ('valence ar maximum value of ',  max_elemVal,max_elemA)
print ('valence ar minimum value of ',  min_elemVal,min_elemA)
print ('valence ar range is ', range_elemVal,range_elemA)
print ('media val e ar  ', mediaVal,mediaA )

The number of valence samples is 13915 13915
valence ar maximum value of  8.53 7.79
valence ar minimum value of  1.26 1.6
valence ar range is  7.27 6.1899999999999995
media val e ar   5.063846927775765 4.210582105641386


In [15]:
#valenceNormalizzata
valenceNormalizzata=[]
for i in valence:
    z=(i-mediaVal)/range_elemVal 
    #print(z)
    valenceNormalizzata.append(z)

In [16]:
#arousal normalizzata
arousalNormalizzata=[]
for i in arousal:
    z=(i-mediaA)/range_elemA
    #print(z)
    arousalNormalizzata.append(z)

In [17]:
#valenceNormalizzata

In [18]:
#arousalNormalizzata

In [19]:
valArous=[]     

In [20]:
for w, v,a in zip(paroleVocabolarioInglese,valenceNormalizzata,arousalNormalizzata):
    #print(v,a)
    valArous.append([w,v,a])

In [21]:
#valArous

In [22]:
#conservo solo parole contenute nel dizionario
valArousRidotto=[]
for w in valArous:
    if w[0] in vocabulary_words:
        #print(w[0])
        #diz={'word':w[0],'valence':w[1],'arousal':w[2]}
        #print(diz)
        valArousRidotto.append(w)
#print(valArousRidotto) #per ogni parola c'è valence e arousual ex (mamma -456, 77)

input_matrix = list()
for member in valArousRidotto:
    input_matrix.append(member[1:])

va_matrix = numpy.array(input_matrix)
print(va_matrix, type(va_matrix), va_matrix.shape)


[[-0.08030907 -0.11479517]
 [ 0.26632092  0.10329853]
 [ 0.21680235 -0.13418128]
 ...
 [-0.04317014 -0.06956092]
 [ 0.26632092  0.22930822]
 [ 0.10951211  0.23738577]] <class 'numpy.ndarray'> (2849, 2)


In [23]:
flattened_va_matrix = va_matrix.flatten(order="F")
print(flattened_va_matrix, flattened_va_matrix.shape)



[-0.08030907  0.26632092  0.21680235 ... -0.06956092  0.22930822
  0.23738577] (5698,)


In [24]:
5698*2

11396

In [25]:
X=[]
#print(flattened_va_matrix)
frasi_bow_np=frasi_BoW.toarray()
for i in frasi_bow_np:
    X.append([*i,*flattened_va_matrix])
#X    
print(numpy.array(X).shape)
    

(4613, 11511)


In [26]:
Y=newArray

In [27]:
>>> from sklearn.model_selection import cross_val_score
>>> from sklearn.tree import DecisionTreeClassifier
>>> clf = DecisionTreeClassifier(random_state=0)
#>>> cross_val_score(clf, X, Y, cv=10)

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [29]:
>>> import numpy as np
>>> from sklearn.datasets import make_multilabel_classification
>>> from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier


In [30]:
X_test_np = np.array( X_test)

In [31]:
y_test_np= np.array( y_test)

In [34]:
from sklearn.neural_network import MLPClassifier
classificatoreMLP = MultiOutputClassifier(MLPClassifier(random_state=1, max_iter=300)).fit(X_train, y_train)
classificatoreMLP.score(X_test_np,y_test_np)

MemoryError: Unable to allocate 8.78 MiB for an array with shape (11511, 100) and data type float64

In [ ]:
classificatore = MultiOutputClassifier(DecisionTreeClassifier(random_state=0)).fit(X_train, y_train)

In [27]:
y_pred = classificatore.predict(X_test)

In [33]:
classificatore.score(X_test_np,y_test_np)

0.07803468208092486

In [32]:
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score

In [29]:

def analizzatore(y_test, y_pred):
    print("explained_variance_score {}".format(explained_variance_score(y_test, y_pred)))
    print("mean_absolute_error {}".format(mean_absolute_error(y_test, y_pred)))
    print("mean_squared_error {}".format(mean_squared_error(y_test, y_pred)))
    print("mean_squared_log_error {}".format(mean_squared_log_error(y_test, y_pred)))
    print("mean_absolute_percentage_error {}".format(mean_absolute_percentage_error(y_test, y_pred)))
    print("median_absolute_error {}".format(median_absolute_error(y_test, y_pred)))
    print("r2_score {}".format(r2_score(y_test, y_pred)))
    
    

In [56]:
analizzatore(y_test, y_pred)

explained_variance_score 0.6606749237394158
mean_absolute_error 0.0005103669371719158
mean_squared_error 0.0005160846378806124
mean_squared_log_error 0.0002450711789701944
mean_absolute_percentage_error 958356564027.0972
median_absolute_error 0.0
r2_score 0.6604212259583525


In [29]:
>>> from sklearn.ensemble import BaggingClassifier
>>> from sklearn.tree import ExtraTreeClassifier
classificatore2 = MultiOutputClassifier(ExtraTreeClassifier(random_state=0)).fit(X_train, y_train)

In [60]:
y_pred_extraTree = classificatore2.predict(X_test)

In [61]:
analizzatore(y_test, y_pred_extraTree)

explained_variance_score 0.5188918256164808
mean_absolute_error 0.0008066929912921904
mean_squared_error 0.0008166368186116626
mean_squared_log_error 0.00038852423914901587
mean_absolute_percentage_error 1701754646403.2566
median_absolute_error 0.0
r2_score 0.518438251512855


In [34]:
classificatore2.score(X_test_np,y_test_np)

0.011560693641618497

In [36]:
from sklearn.ensemble import ExtraTreesClassifier
classificatore3 = MultiOutputClassifier(ExtraTreesClassifier(n_estimators=100, random_state=0)).fit(X_train, y_train)
#y_pred_extraTrees = classificatore3.predict(X_test)
#analizzatore(y_test, y_pred_extraTrees)
classificatore3.score(X_test_np,y_test_np)

0.008670520231213872

In [37]:
from sklearn.neighbors import KNeighborsClassifier
classificatore4 = MultiOutputClassifier(KNeighborsClassifier(n_neighbors=3)).fit(X_train, y_train)
classificatore4.score(X_test_np,y_test_np)

0.015895953757225433

In [38]:
from sklearn.ensemble import RandomForestClassifier
classificatore5 = MultiOutputClassifier(RandomForestClassifier(max_depth=2, random_state=0)).fit(X_train, y_train)
classificatore5.score(X_test_np,y_test_np)

0.000722543352601156

In [39]:
from sklearn.linear_model import RidgeClassifierCV
classificatore6 = MultiOutputClassifier(RidgeClassifierCV(alphas=[1e-3, 1e-2, 1e-1, 1])).fit(X_train, y_train)
classificatore6.score(X_test_np,y_test_np)

KeyboardInterrupt: 

In [ ]:
#codice che non contiene il metodo score()

In [67]:
from sklearn.neighbors import KNeighborsClassifier
classificatore = MultiOutputClassifier(KNeighborsClassifier(n_neighbors=3)).fit(X_train, y_train)
y_pred_KNeighborsClassifier = classificatore.predict(X_test)
analizzatore(y_test, y_pred_KNeighborsClassifier)

explained_variance_score 0.7158673725746472
mean_absolute_error 0.0004892363041180372
mean_squared_error 0.0004966941746076414
mean_squared_log_error 0.000235840101233975
mean_absolute_percentage_error 106919453112.83661
median_absolute_error 0.0
r2_score 0.7154289513916994


In [28]:
from sklearn.ensemble import RandomForestClassifier
classificatore2 = MultiOutputClassifier(RandomForestClassifier(max_depth=2, random_state=0)).fit(X_train, y_train)
y_pred_RandomForestClassifier = classificatore2.predict(X_test)
analizzatore(y_test, y_pred_RandomForestClassifier)

NameError: name 'analizzatore' is not defined

In [30]:
analizzatore(y_test, y_pred_RandomForestClassifier)

explained_variance_score 0.7173576466540512
mean_absolute_error 0.0004845129861412878
mean_squared_error 0.000491970856630892
mean_squared_log_error 0.0002335707688763518
mean_absolute_percentage_error 0.0004812812422624593
median_absolute_error 0.0
r2_score 0.7168757353146658


In [31]:
from sklearn.linear_model import RidgeClassifierCV
classificatore2 = MultiOutputClassifier(RidgeClassifierCV(alphas=[1e-3, 1e-2, 1e-1, 1])).fit(X_train, y_train)
y_pred_RidgeClassifierCV = classificatore2.predict(X_test)
analizzatore(y_test, y_pred_RidgeClassifierCV)

explained_variance_score 0.7198687182389218
mean_absolute_error 0.0004337994668119791
mean_squared_error 0.00043553963659288667
mean_squared_log_error 0.00020763108622921884
mean_absolute_percentage_error 464623801485.1
median_absolute_error 0.0
r2_score 0.7195326764972703
